In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scripts.locator import Locator

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [3]:
df_municipalities = pd.read_csv("../../data/processed/filtered_municipalities.csv")
df_airports = pd.read_csv("../../data/processed/filtered_airports.csv")
df_railway = pd.read_csv("../../data/processed/filtered_trains.csv")

In [4]:
df_municipalities.head()

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.6857,-2.513507,42.84149,1001
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.6910,-3.001022,43.05265,1002
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.8797,-2.566000,43.05400,1003
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.9808,-3.128209,43.12220,1004
4,Álava,Armiñón,247,127,120,arminon,alava,463.5815,-2.872574,42.72305,1006


In [5]:
df_airports.head()

,airport_id,airport_name,latitude,longitude
0,1,Aeropuerto de Jerez,36.743828,-6.062658
1,2,Aeropuerto de Vigo,42.225740,-8.630491
2,3,Aeropuerto de Badajoz,38.889845,-6.821121
3,4,Aeropuerto de Granada - Jaén F.G.L.,37.186826,-3.778613
4,5,Aeropuerto de Almería,36.844781,-2.371234


In [6]:
df_railway.sample(15)

,station_code,station_name,latitude,longitude,province_clean,municipality_clean
536,10503,Valdestillas,41.488726,-4.772956,valladolid,valdestillas
176,5000,Granada,37.184036,-3.609036,granada,granada
457,71305,Duesaigües-L'Argentera,41.143639,0.920835,tarragona,"argentera, l'"
515,60206,Tembleque,39.722462,-3.497034,toledo,tembleque
162,79205,Fornells De La Selva,41.935026,2.809754,girona,fornells de la selva
319,61012,Cieza,38.246810,-1.422045,murcia,cieza
293,22004,San Pedro Do Sil,42.452828,-7.712340,lugo,panton
517,60301,Quero,39.504187,-3.266023,toledo,quero
227,56004,Jodar-Ubeda,37.913342,-3.350202,jaen,jodar
465,76004,Valls,41.289585,1.258754,tarragona,valls


In [7]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_municipalities[['closest_distance_airport', 'nearest_airport']] = Locator(df_municipalities, df_airports, 'airport_name').result
df_municipalities[['closest_distance_train', 'nearest_train_station']]  = Locator(df_municipalities, df_railway, 'station_name').result

In [10]:
df_accessibility = df_municipalities[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]

In [11]:
# df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
# df_accessibility = pd.read_csv("../../data/scored/accessibility.csv")
df_hospitals = pd.read_csv("../../data/processed/filtered_hospitals.csv")

In [12]:
df_accessibility = df_accessibility[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]
df_hospitals_general = df_hospitals.query("type == 'General'")

# Apply the function to each row in the towns DataFrame and create two new columns
df_accessibility[['closest_distance_hospital', 'nearest_hospital']] = Locator(df_accessibility, df_hospitals_general, 'hospital_name').result

In [13]:
df_accessibility.to_csv("../../data/processed/processed_distances.csv", index=False)